# Dislocation monopoles

This Notebook follows the workflow for generating dislocation monopoles and analyzing the resulting structure using different techniques.

__Global workflow order:__ 

This Notebook requires

- The "unique_crystals.csv" file generated by the "3. Crystal relaxation" Notebook.

- calculation_elastic_constants_static records generated by the "4. Static elastic constants" Notebook.

**Library imports**

In [1]:
# Standard Python libraries
from __future__ import (absolute_import, print_function,
                        division, unicode_literals)

# http://www.numpy.org/
import numpy as np

from IPython.core.display import display, HTML

# https://pandas.pydata.org/
import pandas as pd

from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

iprPy version 0.8.3


## 0. Access database

### Load database

In [2]:
database = iprPy.load_database('demo')

### Load unique crystals

In [3]:
unique_crystals = pd.read_csv('unique_crystals.csv')
print(len(unique_crystals))

44


### Specify prototype-composition delimiters

In [4]:
proto_comps = {}
proto_comps['A1--Cu--fcc'] = ['Ne', 'Al', 'AlS', 'Ar', 'Ca', 'Fe', 'FeS', 'Ni', 'Cu', 'CuS', 
                              'Kr', 'Sr', 'Rh', 'Pd', 'Ag', 'Xe', 'Ir', 'Pt', 'Au', 'Pb', 
                              'Ac', 'Ce', 'Yb', 'Th']
proto_comps['A2--W--bcc'] = ['Li', 'Na', 'K', 'V', 'Cr', 'Fe', 'FeS', 'Rb', 'Nb', 'Mo', 
                             'Cs', 'Ba', 'Ta', 'W', 'Eu']
proto_comps['A3--Mg--hcp'] = ['H', 'He', 'Be', 'Mg', 'MgS', 'Sc', 'Ti', 'Co', 'Zn', 'Y', 
                              'Zr', 'Tc', 'Ru', 'Cd', 'Hf', 'Re', 'Os' 'Tl', 'Gd', 'Tb', 
                              'Dy', 'Ho', 'Er', 'Tm', 'Lu']
proto_comps['A4--C--dc'] = ['C', 'Si', 'SiS', 'Ge', 'Sn']

### Limit unique crystals using prototype-composition delimiters

In [5]:
proto_parents = {}
unique_crystal_keys = []
for prototype in proto_comps:
    proto_parents[prototype] = unique_crystals[(unique_crystals.prototype == prototype)
                                              &(unique_crystals.composition.isin(proto_comps[prototype]))].calc_key.tolist()
    print(len(proto_parents[prototype]), prototype)
    unique_crystal_keys.extend(proto_parents[prototype])
print(len(unique_crystal_keys), 'total')

2 A1--Cu--fcc
0 A2--W--bcc
0 A3--Mg--hcp
0 A4--C--dc
2 total


## 1. point_defect_static calculation

In [6]:
calculation = iprPy.load_calculation('dislocation_monopole')
run_directory = iprPy.load_run_directory('demo_4')

### Build input_dict

In [7]:
print(calculation.allkeys)

['lammps_command', 'mpi_command', 'length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'dislocation_boundarywidth', 'dislocation_boundaryshape', 'potential_file', 'potential_content', 'potential_dir', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'elasticconstants_file', 'elasticconstants_content', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C22', 'C23', 'C24', 'C25', 'C26', 'C33', 'C34', 'C35', 'C36', 'C44', 'C45', 'C46', 'C55', 'C56', 'C66', 'sizemults', 'dislocation_file', 'dislocation_content', 'dislocation_family', 'dislocation_burgersvector', 'dislocation_stroh_m', 'dislocation_stroh_n', 'dislocation_lineboxvector', 'a_uvw', 'b_uvw', 'c_uvw', 'atomshift', 'randomseed', 'annealtemperature', 'energytolerance', 'forcetolerance', 'maxiterations', 'maxevaluations', 'maxatommotion']


In [8]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 c:\\Program Files\\MPICH2\\bin\\mpiexec -localonly 4

# Build load information from crystal_space_group results
buildcombos                 atomicarchive load_file parent
parent_record               calculation_elastic_constants_static
parent_load_key             system-info
parent_strainrange          1e-7

# Build defect records
buildcombos                 defect dislocation_file
defect_record               dislocation_monopole
#defect_id                   A1--Cu--fcc--a-2-110--0-screw--{111}

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                 
atomshift                   
sizemults                   10 10 10

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
dislocation_boundarywidth   
dislocation_boundaryshape
annealtemperature
randomseed
energytolerance             
forcetolerance              
maxiterations               
maxevaluations              
maxatommotion               
"""

with open('input_script.in', 'w') as f:
    f.write(input_script)

### Parse input script

In [9]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)

### Add parent_load_file list using unique crystals

In [10]:
input_dict['parent_load_file'] = []
for key in unique_crystal_keys:
    input_dict['parent_load_file'].append(key + '.json')

In [11]:
input_dict

{'lammps_command': 'lmp_mpi',
 'mpi_command': 'c:\\Program Files\\MPICH2\\bin\\mpiexec -localonly 4',
 'buildcombos': ['atomicarchive load_file parent', 'defect dislocation_file'],
 'parent_record': 'calculation_elastic_constants_static',
 'parent_load_key': 'system-info',
 'parent_strainrange': '1e-7',
 'defect_record': 'dislocation_monopole',
 'sizemults': '10 10 10',
 'parent_load_file': ['3ba5fd68-696a-4543-944b-793db7b31f2d.json',
  '37e23240-e890-41ac-b103-f716235b586f.json']}

### Prepare calculations

In [12]:
database.get_records_df(style='dislocation_monopole', id='A1--Cu--fcc--a-2-110--0-screw--{111}')

,a_uvw,atomshift,b_uvw,burgersstring,burgersvector,c_uvw,character,family,id,key,lineboxvector,linedirection,slipplane,stroh_m,stroh_n
0,1 -1 0,0.00 0.01 0.25,1 1 -2,"a/2[ 1,-1, 0]",0.5 -0.5 0.0,1 1 1,screw,A1--Cu--fcc,A1--Cu--fcc--a-2-110--0-screw--{111},fbb42787-440e-46cb-bce3-584857257907,a,"[1, -1, 0]","[1, 1, 1]",0 1 0,0 0 1


In [14]:
database.prepare(run_directory, calculation, **input_dict)

elasticconstants_file


AssertionError: 

In [ ]:
database.check_records(calculation.record_style)

In [ ]:
killit

### Run calculations

In [ ]:
database.runner(run_directory)

In [ ]:
results_df = database.get_records_df(style=calculation.record_style)
error_df = results_df[results_df.status=='error']
print(len(error_df), 'calculations issued errors:')
for error in np.unique(error_df.error):
    print(error)

## 2. Calculation analysis

In [ ]:
results = database.get_records_df(style=calculation.record_style)

In [ ]:
# Compute surface energy
energy_units = 'eV'
results = results[results.reconfigured == False]
results['Ef ('+energy_units+')'] = uc.get_in_units(results.E_f, energy_units)
iprPy.analysis.assign_composition(results, database)

### Display results

In [ ]:
# Display results
potential_LAMMPS_id = '1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1'
composition = 'Ni'

display_results = results[(results.potential_LAMMPS_id == potential_LAMMPS_id)
                         &(results.composition == composition)]
display_results = display_results[['Ef ('+energy_units+')', 'pointdefect_id']].sort_values('Ef ('+energy_units+')')

print('Point defect formation energies for', composition, 'using', potential_LAMMPS_id)
display(HTML(display_results.to_html(index=False)))